In [1]:
import gurobipy as gp
import numpy as np

In [5]:
# A1 = np.array([[2, 1], [1, 2]])
# print(f'A1 eigvals: {np.linalg.eigvals(A1)}')
# A2 = np.array([[1.5, 1], [1, 1.5]])
# print(f'A2 eigvals: {np.linalg.eigvals(A2)}')
# B = np.array([[3.5, 2], [2, 3.5]])
# print(f'B eigvals:  {np.linalg.eigvals(B)}')
# print(f'A1 + A2 - B eigvals: {np.linalg.eigvals(A1 + A2 - B)}')

A1 = np.array([[4, 0], [0, 3]])
print(f'A1 eigvals: {np.linalg.eigvals(A1)}')
A2 = np.array([[2, 0], [0, 1]])
print(f'A2 eigvals: {np.linalg.eigvals(A2)}')
B = np.array([[5, 0], [0, 3]])
print(f'B eigvals:  {np.linalg.eigvals(B)}')
print(f'A1 + A2 - B eigvals: {np.linalg.eigvals(A1 + A2 - B)}')

A1 eigvals: [3. 2.]
A2 eigvals: [2. 1.]
B eigvals:  [5. 3.]
A1 + A2 - B eigvals: [0. 0.]


In [3]:
m = gp.Model()
z = m.addMVar((2,), lb=-1, ub=1)

# constraints for sum of norms
A1z = m.addMVar((2,))
A2z = m.addMVar((2,))
Bz = m.addMVar((2,))
m.addConstr(A1z == A1 @ z)
m.addConstr(A2z == A2 @ z)
m.addConstr(Bz == B @ z)
norm_A1z = m.addVar()
norm_A2z = m.addVar()
norm_Bz = m.addVar()
m.addGenConstrNorm(norm_A1z, A1z, 2, "FuncPieces=100000")
m.addGenConstrNorm(norm_A2z, A2z, 2, "FuncPieces=100000")
m.addGenConstrNorm(norm_Bz, Bz, 2, "FuncPieces=100000")

# solve opt problem
# m.addConstr(norm_A1z + norm_A2z - norm_Bz >= 1e-3, "FuncPieces=10000")
obj = norm_A1z + norm_A2z - norm_Bz
m.setObjective(obj, sense=gp.GRB.MAXIMIZE)
# m.setObjective(0.0)
m.setParam('LogToConsole', 0)
m.setParam('NonConvex', 2)
m.setParam('FeasibilityTol', 1e-9)
m.setParam('OptimalityTol', 1e-9)
m.setParam('Presolve', 0)
m.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-02


In [4]:
print(f'model status: {m.status}')

print(f'objective value: {obj.getValue()}')
print(f'optimal solution: {z.X}')
obj1 = norm_A1z.X + norm_A2z.X - norm_Bz.X
obj2 = np.linalg.norm(A1 @ z.X) + np.linalg.norm(A2 @ z.X) - np.linalg.norm(B @ z.X)
print(f'objective values: {obj1}, {obj2}')

z = np.ones(2)
value = np.linalg.norm(A1 @ z) + np.linalg.norm(A2 @ z) - np.linalg.norm(B @ z)
print(f'with z = {z}, ||A1@z|| + ||A2@z|| - ||B@z|| = {value}')

model status: 2
objective value: 0.010905635586355267
optimal solution: [0.8295364 1.       ]
objective values: 0.010905635586355267, 0.010905635525850776
with z = [1. 1.], ||A1@z|| + ||A2@z|| - ||B@z|| = 0.010667358118478631
